## Rat R

Define functions

In [ ]:
import sys
dir1 = '/Users/erihe/OneDrive - NTNU/'
if not dir1 in sys.path: sys.path.append(dir1)
from utils_new import *





def get_pos(x, y, t, hd, dt_orig = 0.01, res = 100000, min_time = -1, max_time = None):
    dt = int(dt_orig*res)
    if min_time<0:
        min_time = np.floor(t[0])*res-dt
    else:
        min_time*=res
    if not max_time:
        max_time = np.ceil(t[-1])*res+dt
    else:
        max_time*=res

    for arr in [x,y,hd]:
        if len(arr)>0:
            arr[np.isnan(arr)] = np.mean(arr[~np.isnan(arr)])
#        fill_nans(arr)

    tt = np.arange(np.floor(min_time), np.ceil(max_time)+1, dt)/res
    xspline = CubicSpline(t, x)
    xx = xspline(tt)
    if len(y) > 0:
        xx = gaussian_filter1d(x, 100)
        xspline = CubicSpline(t, xx)
        xx = xspline(tt)
        yy = gaussian_filter1d(y, 100)
        yspline = CubicSpline(t, yy)
        yy = yspline(tt)
        speed = np.sqrt(np.square(np.diff(xx)) + 
                        np.square(np.diff(yy)))
#        speed = np.sqrt(np.square(np.diff(gaussian_filter1d(xx, 100))) + 
#                        np.square(np.diff(gaussian_filter1d(yy, 100))))
    else:
        yy = []
        speed = np.square(np.diff(xx))
    speed = np.concatenate(([speed[0]],speed))/dt_orig
    
    if len(hd)>0:
        hdcosspline = CubicSpline(t, np.cos(hd))
        hdsinspline = CubicSpline(t, np.sin(hd))
        aa = np.arctan2(hdsinspline(tt),hdcosspline(tt))%(2*np.pi)
    else:
        aa = []
    return tt, xx, yy, speed, aa    

Load data 

In [ ]:
data_dir = '/Users/erihe/OneDrive - NTNU/Prosjekt/Main/'

In [ ]:
f = np.load(data_dir + 'is_conjunctive_all.npz', allow_pickle = True)
isall = {}
for k in list(f.keys()):
    print(k, np.bincount(f[k]))
    isall[k] = f[k]
f.close()

Muas etc retyped in Matlab for easy python interpretation 

In [ ]:
data_dir = '/Users/erihe/OneDrive - NTNU/Prosjekt/'
f = np.load(data_dir + '/Toroidal_topology_grid_cell_data/rat_r_day1_grid_modules_1_2_3.npz', 
            allow_pickle = True)
spikes = {}
it = 0
for k in ['spikes_mod1','spikes_mod2','spikes_mod3']:
    spktemp = f[k][()]
    for kk in spktemp:
        spikes[it] = spktemp[kk]
        it += 1
t = f['t']
x = f['x']
y = f['y']
hd_azimuth = f['azimuth']
f.close()

In [ ]:
min_of_1, max_of_1 = 7457, 14778
min_of_2, max_of_2 = 14890, 16045
min_w_1, max_w_1 = 16925, 18026
min_w_2, max_w_2 = 18183, 20704
min_w_3, max_w_3 = 20895, 23771


In [ ]:
num_neurons = len(spikes)
meanRate_of = np.zeros((num_neurons))
for i in range(num_neurons):
    spk = spikes[i].copy()
    meanRate_of[i] = np.sum((spk>min_of_1) & (spk<max_of_1))
meanRate_of/=(max_of_1-min_of_1)


Compute firing rates for open field session and cross correlations, 
 
 
NOTE: Could have memory issues, change either session duration, time interval (dt_orig) or time resolution of the bin depiction (res).

In [ ]:
t0 = time.time()
sspikes_of1, __ = firing_rate(spikes, sigma = 1, min_time = min_of_1, max_time = max_of_1, 
                             dt_orig = 0.1, res = 10000)
sspikes_of1 = np.sqrt(sspikes_of1)
print(time.time()- t0)


t0 = time.time()
sspikes_of2, __ = firing_rate(spikes, sigma = 1, min_time = min_of_2, max_time = max_of_2, 
                             dt_orig = 0.1, res = 10000)
sspikes_of2 = np.sqrt(sspikes_of2)
print(time.time()- t0)

sspikes_of1 = np.concatenate((sspikes_of1,sspikes_of2),0)


t0 = time.time()
Xcorr_of1 =  cross_corr_dist(sspikes_of1, lencorr = 30)
print(time.time()-t0)



In [ ]:
t0 = time.time()
sspikes_w1, __ = firing_rate(spikes, sigma = 1, min_time = min_w_1, max_time = max_w_1, 
                             dt_orig = 0.1, res = 10000)
sspikes_w1 = np.sqrt(sspikes_w1)
print(time.time()- t0)


t0 = time.time()
sspikes_w2, __ = firing_rate(spikes, sigma = 1, min_time = min_w_2, max_time = max_w_2, 
                             dt_orig = 0.1, res = 10000)
sspikes_w2 = np.sqrt(sspikes_w2)
print(time.time()- t0)
sspikes_w1 = np.concatenate((sspikes_w1,sspikes_w2),0)


t0 = time.time()
Xcorr_w1 =  cross_corr_dist(sspikes_w1, lencorr = 30)
print(time.time()-t0)



In [ ]:

t0 = time.time()
sspikes_w3, __ = firing_rate(spikes, sigma = 1, min_time = min_w_3, max_time = max_w_3, 
                            dt_orig = 0.1, res = 10000)
sspikes_w3 = np.sqrt(sspikes_w3)
print(time.time()- t0)


t0 = time.time()
Xcorr_w3 =  cross_corr_dist(sspikes_w3, lencorr = 30)
print(time.time()-t0)


In [ ]:
np.savez('R1_Xcorrs',Xcorr_of1 = Xcorr_of1, Xcorr_w1 = Xcorr_w1 ,Xcorr_w3 = Xcorr_w3)


In [ ]:
f = np.load('R1_Xcorrs.npz', allow_pickle = True)
Xcorr_of1 = f['Xcorr_of1'] 
Xcorr_w1 = f['Xcorr_w1']
Xcorr_w3 = f['Xcorr_w3']
f.close()


In [ ]:
tt, xx, yy, speed, hd_of = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_of_1, max_time = max_of_1, dt_orig = 0.1, res = 10000)

tt0, xx0, yy0, speed0, hd_of0 = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_of_2, max_time = max_of_2, dt_orig = 0.1, res = 10000)


tt, xx, yy, speed, hd_of = (np.concatenate((tt,tt0)), 
                                 np.concatenate((xx,xx0)), 
                                 np.concatenate((yy,yy0)), 
                                 np.concatenate((speed, speed0)),
                                 np.concatenate((hd_of,hd_of0)))

In [ ]:
tt1, xx1, yy1, speed1, hd_of1 = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_of_1, max_time = max_of_1, dt_orig = 0.01, res = 100000)


In [ ]:

hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    mtot, __,  circ  = binned_statistic(hd_of, sspikes_of1[:,i],
                                              statistic = 'mean', 
                                              bins = 30)
    mu = np.mean(sspikes_of1[:,i])
    hd_info[i] = information_score_1d(mtot, circ-1, mu)


Cluster neurons using agglomerative clustering with average linkage, of given threshold.

In [ ]:
d1 = np.zeros((len(spikes),len(spikes)))
for x in [Xcorr_w1, Xcorr_w3, Xcorr_of1  ]:    
    x[np.isnan(x)] = 0
    dd1 = squareform(pdist(np.square(x), 'correlation'))
    dd1[np.isnan(dd1)] = 2
    d1 += dd1
infs = np.isinf(d1)
d1[infs] = np.max(d1[~infs])*2
d1 /= 3

In [ ]:
is_conj_all = np.concatenate((isall['is_conj_R1_day1'],
                              isall['is_conj_R2_day1'],
                              isall['is_conj_R3_day1']))

In [ ]:
is_conj_all1 = np.zeros(len(is_conj_all), dtype = int)
len1 = len(isall['is_conj_R1_day1'])
len3 = len(isall['is_conj_R3_day1'])
is_conj_all1[:len1][isall['is_conj_R1_day1']] = 1
is_conj_all1[:len1][~isall['is_conj_R1_day1']] = 2
is_conj_all1[len1:-len3][isall['is_conj_R2_day1']] = 3
is_conj_all1[len1:-len3][~isall['is_conj_R2_day1']] = 4
is_conj_all1[-len3:][isall['is_conj_R3_day1']] = 5
is_conj_all1[-len3:][~isall['is_conj_R3_day1']] = 6
is_conj_all1 -= 1

In [ ]:
plt.viridis()
dvals = d1[np.triu_indices(len(d1),1)]
thr = np.round(np.percentile(dvals,42),1)
thr = 0.4
print(thr)
ind1 = get_ind(d1,thr, linkage = 'average')
indcount = np.bincount(ind1)
rel_inds = np.where(indcount>20)[0]
for i in rel_inds:
    print(i)
    print(np.bincount(is_conj_all1[ind1 == i], minlength = 6))
    print(np.bincount(is_conj_all1[ind1 == i], minlength = 6)/indcount[i])

In [ ]:

dim = 8         # number of principal components
k = 1000          # number of neighbours for downsampling
nbs = 1000         # number of neighbours for metric
maxdim = 1        # dimension of homology - often just do 1 as it could be expensive (depends on number of points and neighbours)
metric = 'cosine' # what metric to use for persistence
pthr = 0
dt_curr = 0.01
time_resolution = 100000

sigma = 6
sp = 0.1
eps = 0.8
nn = 2
n_points = 2200
sspikes_of = firing_rate(spikes, sigma = sigma, min_time = min_of_1, max_time = max_of_1, 
                          dt_orig = dt_curr, res = time_resolution)[0]

i = np.argmax(np.bincount(ind1))
mod_ind1s = np.where(ind1 == i)[0]            
##########    Initial preprocessing ########## 
sspikes1 = np.sqrt(sspikes_of[:, mod_ind1s])
num_times_all, num_neurons = np.shape(sspikes1)

movetimes0 = np.arange(0,num_times_all,nn)
movetimes0 = movetimes0[np.where((speed1[movetimes0]>=0.1))[0]]
print('len movetimes0', len(movetimes0))
spksum = np.sum(sspikes1,1)[movetimes0]
movetimes0 = movetimes0[np.where(spksum> np.percentile(spksum,pthr))[0]]
sspikes1 = sspikes1[movetimes0, :]

##########    Dimension reduce ##########            
dim_red_spikes_move_scaled = preprocessing.scale(sspikes1,axis = 0)
dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])

fig, ax = plt.subplots(1,1)
ax.plot(var_exp[:15])
ax.set_aspect(1/ax.get_data_ratio())
plt.show()

##########    Downsample ##########            
startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled),1))
movetimes1 = radial_downsampling(dim_red_spikes_move_scaled, epsilon = eps, 
    startindex = startindex)
print('len movetimes1', len(movetimes1))

indstemp  = sample_denoising(dim_red_spikes_move_scaled[movetimes1,:],  k, 
                                   n_points, 1, 'cosine')[0]
indstemp = movetimes1[indstemp]
dim_red_spikes_move_scaled = dim_red_spikes_move_scaled[indstemp,:]
np.savez('roger_conj', movetimes0 = movetimes0, movetimes1 = movetimes1, indstemp = indstemp)

indstemp = indstemp[:n_points]
dim_red_spikes_move_scaled = dim_red_spikes_move_scaled[:n_points,:]

X = squareform(pdist(dim_red_spikes_move_scaled[:,:], metric))

##########    Compute cohomology ##########            
persistence, dgms = compute_persistence(X, maxdim)
plt.figure()
plot_diagrams(dgms, list(np.arange(maxdim+1)), lifetime = True)
plt.show()
fig = plot_barcode(dgms)

### UMAP

In [ ]:
from umap import UMAP

help(UMAP)

uclass = UMAP(n_neighbors=100, 
             n_components=3, 
             metric='cosine', 
             metric_kwds=None, 
             output_metric='euclidean', 
             output_metric_kwds=None, 
             n_epochs=None, 
             learning_rate=1.0, 
             init='spectral',
              min_dist=0.1, 
              spread=1.0, low_memory=True, 
         n_jobs=-1, set_op_mix_ratio=1.0, local_connectivity=1.0, 
         repulsion_strength=1.0, negative_sample_rate=5, transform_queue_size=4.0, a=None, b=None,
         random_state=None, angular_rp_forest=False, target_n_neighbors=-1, target_metric='categorical', 
         target_metric_kwds=None, target_weight=0.5, transform_seed=42, transform_mode='embedding', 
         force_approximation_algorithm=False, verbose=False, tqdm_kwds=None, unique=False, densmap=False,
         dens_lambda=2.0, dens_frac=0.3, dens_var_shift=0.1, output_dens=False, disconnection_distance=None, 
         precomputed_knn=(None, None, None))

In [ ]:
n_points = 2500
sspikes1 = np.sqrt(sspikes_of)
                
randints = np.random.randint(low = 0, high = len(sspikes1), size =10000)
dim_red_spikes_move_scaled = preprocessing.scale(sspikes1[randints,:],axis = 0)
dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])

In [ ]:
e4 = uclass.fit_transform(dim_red_spikes_move_scaled)


In [ ]:
for i in [-60, 0, 60]:
    for j in [-120, -60, 0, 60, 120]:    
        plt.hsv()
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(e4[:,0], e4[:,1], e4[:,2], s = 4, c = hd_of1[randints])
        ax.view_init(i,j)
        ax.axis('off')

In [ ]:
sspikes1 = np.sqrt(sspikes_of)
                
dim_red_spikes_move_scaled = preprocessing.scale(sspikes1[:, mod_ind1s][movetimes0,:],axis = 0)
dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])


In [ ]:
e4 = uclass.transform(dim_red_spikes_move_scaled[movetimes1])


In [ ]:
for i in [-60, 0, 60]:
    for j in [ -120, -60, 0, 60, 120]:   
        print(i,j)
        plt.hsv()
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(e4[:,0], e4[:,1], e4[:,2], s = 4, c = hd_of1[movetimes0][movetimes1])
        ax.view_init(i,j)
        ax.axis('off')
        plt.show()

### Spike train

In [ ]:
min_time = min_of_1#np.min(t)
max_time = max_of_1 #np.max(t)

lent = max_time-min_time
lent = 100
t0 = min_time+500
t1 = t0 + lent
fig, ax = plt.subplots(1,1,figsize = (14, 8), dpi = 620)
spk = []
for i in spikes:
    spktmp = spikes[i].copy()
    spk.append(spktmp[(spktmp<t1) & (spktmp>t0)])
ax.eventplot(spk, color = 'gray', linelengths =0.4)#, color=colorCodes, linelengths = lineSize)  

#plt.plot([0,lent], [-1,-1], c = 'k')
ax.set_xlim([t0,t1])
ax.set_ylim([0,len(spk)])
ax.set_xticks([t0,t1])
ax.set_yticks([0,len(spk),])
ax.set_xticklabels('')
ax.set_yticklabels('')
#ax.set_xlabel('Time (' + str(int(lent)) + 's)')
#ax.set_ylabel('Neurons ' + str(len(spikes)))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(ff)
t0 = t1
plt.show()


In [ ]:
min_time = min_of_1#np.min(t)
max_time = max_of_1 #np.max(t)

lent = max_time-min_time
lent = 100
t0 = min_time+500
t1 = t0 + lent
fig, ax = plt.subplots(1,1,figsize = (14, 8), dpi = 620)
spk = []
for i in mod_ind1s:
    spktmp = spikes[i].copy()
    spk.append(spktmp[(spktmp<t1) & (spktmp>t0)])
ax.eventplot(spk, color = 'gray', linelengths =0.4)#, color=colorCodes, linelengths = lineSize)  

#plt.plot([0,lent], [-1,-1], c = 'k')
ax.set_xlim([t0,t1])
ax.set_ylim([0,len(spk)])
ax.set_xticks([t0,t1])
ax.set_yticks([0,len(spk),])
ax.set_xticklabels('')
ax.set_yticklabels('')
#ax.set_xlabel('Time (' + str(int(lent)) + 's)')
#ax.set_ylabel('Neurons ' + str(len(spikes)))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(ff)
t0 = t1
plt.show()


In [ ]:
78/140*2.86
tt1_ds = tt1[movetimes0[indstemp]]
min_time = min_of_1+500#np.min(t)
max_time = max_of_1 #np.max(t)

lent = max_time-min_time
lent = 100
t0 = min_time
t1 = t0 + lent
tt1_ds1 = tt1_ds[(tt1_ds>t0) & (tt1_ds<t1)]
fig, ax = plt.subplots(1,1,figsize = (14, 8), dpi = 620)
spk = []
for i in mod_ind1s:    
    spktmp = spikes[i].copy()
    spktmp = spktmp[(spktmp<t1) & (spktmp>t0)]
    spktmp1 = []
    for ii, i in enumerate(tt1_ds1):
        ss1 = spktmp-i
        if np.sum((ss1<=0.01) & (ss1>0)):
            spktmp1.extend([i])
    spk.append(spktmp1)
ax.eventplot(spk, color = 'gray', linelengths =3, linewidths = 5)#, color=colorCodes, linelengths = lineSize)  

#t0 = 0
#t1 = len(tt1_ds1)
#plt.plot([0,lent], [-1,-1], c = 'k')
ax.set_xlim([t0,t1])
ax.set_ylim([0,len(spk)])
ax.set_xticks([t0,t1])
ax.set_yticks([0,len(spk),])
ax.set_xticklabels('')
ax.set_yticklabels('')
#ax.set_xlabel('Time (' + str(int(lent)) + 's)')
#ax.set_ylabel('Neurons ' + str(len(spk)))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.title(ff)
t0 = t1
plt.show()


### CEBRA

In [ ]:
from cebra import CEBRA
batch_size = 128
single_cebra_model = CEBRA(model_architecture = 'offset1-model', 
                           batch_size=batch_size, output_dimension=3,
                           verbose = True, distance = 'cosine', 
                          )
single_cebra_model_euclidean = CEBRA(model_architecture = 'offset1-model-mse', 
                           batch_size=batch_size, output_dimension=3,
                           verbose = True, distance = 'euclidean'
                          )


In [ ]:
randints = np.random.randint(low = 0, high = len(sspikes1), size =len(movetimes0))
dim_red_spikes_move_scaled = preprocessing.scale(sspikes1[randints,:], axis = 0)
dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])

In [ ]:
single_cebra_model.fit(dim_red_spikes_move_scaled[movetimes1])

In [ ]:
single_cebra_model_euclidean.fit(dim_red_spikes_move_scaled[movetimes1])

In [ ]:
cebra_dec = single_cebra_model.transform(dim_red_spikes_move_scaled)

In [ ]:
cebra_dec_euclidean = single_cebra_model_euclidean.transform(dim_red_spikes_move_scaled)

In [ ]:

plt.hsv()
for ii in [-90, -45,0, 45, 90]:
    for jj in [-180, -135, -90, -45,0, 45, 90, 135,180]:
        print(ii,jj)
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(cebra_dec[:,0], cebra_dec[:,1], cebra_dec[:,2], s = 5, c = hd_of1[randints])
        ax.view_init(ii,jj)
        ax.axis('off')
        plt.show()

In [ ]:

plt.hsv()
for ii in [-90, -45,0, 45, 90]:
    for jj in [-180, -135, -90, -45,0, 45, 90, 135,180]:
        print(ii,jj)
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(cebra_dec_euclidean[:,0], cebra_dec_euclidean[:,1], cebra_dec_euclidean[:,2], s = 5, c = hd_of1[randints])
        ax.view_init(ii,jj)
        ax.axis('off')
        plt.show()

In [ ]:
from cebra import CEBRA
batch_size = 128
single_cebra_model = CEBRA(model_architecture = 'offset1-model', 
                           batch_size=batch_size, output_dimension=3,
                           verbose = True, distance = 'cosine', 
                          )
single_cebra_model_euclidean = CEBRA(model_architecture = 'offset1-model-mse', 
                           batch_size=batch_size, output_dimension=3,
                           verbose = True, distance = 'euclidean'
                          )


In [ ]:
n_points = 2500
dim_red_spikes_move_scaled = preprocessing.scale(sspikes1[:, mod_ind1s][movetimes0,:], axis = 0)
dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])

In [ ]:
single_cebra_model.fit(dim_red_spikes_move_scaled[movetimes1])

In [ ]:
single_cebra_model_euclidean.fit(dim_red_spikes_move_scaled[movetimes1])

In [ ]:
cebra_dec = single_cebra_model.transform(dim_red_spikes_move_scaled[movetimes1])

In [ ]:
cebra_dec_euclidean = single_cebra_model_euclidean.transform(dim_red_spikes_move_scaled[movetimes1])

In [ ]:

plt.hsv()
for ii in [-90, -45,0, 45, 90]:
    for jj in [-180, -135, -90, -45,0, 45, 90, 135,180]:
        print(ii,jj)
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(cebra_dec[:,0], cebra_dec[:,1], cebra_dec[:,2], s = 5, c = hd_of1[movetimes0][movetimes1])
        ax.view_init(ii,jj)
        ax.axis('off')
        plt.show()

In [ ]:

plt.hsv()
for ii in [-90, -45,0, 45, 90]:
    for jj in [-180, -135, -90, -45,0, 45, 90, 135,180]:
        print(ii,jj)
        fig = plt.figure(figsize = (10,10), dpi = 120)
        ax = fig.add_subplot(111, projection = '3d')
        ax.scatter(cebra_dec_euclidean[:,0], cebra_dec_euclidean[:,1], cebra_dec_euclidean[:,2], s = 5, c = hd_of1[movetimes0][movetimes1])
        ax.view_init(ii,jj)
        ax.axis('off')
        plt.show()

In [ ]:
coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1,2], 
                bConsistent = True)


In [ ]:

##########    Get coordinates all   ##########            
coords_mod1 = get_coords_all(np.sqrt(sspikes_of[:, ind1==np.argmax(np.bincount(ind1))]),
                             coords_ds_consistent,
                             movetimes0,                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)
coords_mod1 = coords_mod1%(2*np.pi)

In [ ]:


for c in [0,1,2]:
    fig, axs = plt.subplots(1,1)
    nans0 = ~np.isnan(coords_mod1[:,c])
    mtot, __, __, circ  = binned_statistic_2d(xx1,
                                              yy1,
                                              coords_mod1[:,c], 
                                              statistic = circmean, 
                                              bins = 50,
                                              expand_binnumbers = True)

    nans = np.isnan(mtot)
    sintot = np.sin(mtot)
    costot = np.cos(mtot)
    sintot[nans] = np.mean(sintot[~nans])
    costot[nans] = np.mean(costot[~nans])
    sintot = gaussian_filter(sintot,1)
    costot = gaussian_filter(costot,1)
    mtot = np.cos(np.arctan2(sintot, costot))
    mtot = gaussian_filter(mtot,1)
    plt.viridis()
    mtot[nans] = np.nan
    axs.imshow(mtot)
    axs.axis('off')
    axs.set_aspect(1/axs.get_data_ratio())





In [ ]:
coords_mod2 = coords_mod1.copy()
for c in [0,1,2]:
    coords_mod2tmp = 2*np.pi-coords_mod2[:,c]                        
    dstmp = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod2tmp[movetimes0])), 
                    np.mean(np.cos(hd_of1[movetimes0]-coords_mod2tmp[movetimes0])))
    coords_mod2tmp = (coords_mod2tmp+dstmp)%(2*np.pi)


    ds = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod2[movetimes0,c])), 
                    np.mean(np.cos(hd_of1[movetimes0]-coords_mod2[movetimes0,c])))
    coords_mod2[:,c] = (coords_mod2[:,c]+ds)%(2*np.pi)

    ds0 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod2tmp[movetimes0]), 
                          np.cos(hd_of1[movetimes0]-coords_mod2tmp[movetimes0]))))
    ds1 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod2[movetimes0,c]), 
                          np.cos(hd_of1[movetimes0]-coords_mod2[movetimes0,c]))))

    if ds0 < ds1:
        coords_mod2[:,c] = coords_mod2tmp
        print(ds0)
    else:
        print(ds1)                  

In [ ]:
for sig in [5,6,7,8,9,10, 15]:
    coords_mod2 =  np.arctan2(gaussian_filter1d(np.sin(coords_mod1), sigma = sig, axis = 0),
                              gaussian_filter1d(np.cos(coords_mod1), sigma = sig, axis = 0))%(2*np.pi)

    for c in [0,1,2]:
        coords_mod2tmp = 2*np.pi-coords_mod2[:,c]                        
        dstmp = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod2tmp[movetimes0])), 
                        np.mean(np.cos(hd_of1[movetimes0]-coords_mod2tmp[movetimes0])))
        coords_mod2tmp = (coords_mod2tmp+dstmp)%(2*np.pi)


        ds = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod2[movetimes0,c])), 
                        np.mean(np.cos(hd_of1[movetimes0]-coords_mod2[movetimes0,c])))
        coords_mod2[:,c] = (coords_mod2[:,c]+ds)%(2*np.pi)

        ds0 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod2tmp[movetimes0]), 
                              np.cos(hd_of1[movetimes0]-coords_mod2tmp[movetimes0]))))
        ds1 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod2[movetimes0,c]), 
                              np.cos(hd_of1[movetimes0]-coords_mod2[movetimes0,c]))))

        if ds0 < ds1:
            coords_mod2[:,c] = coords_mod2tmp
            print(ds0)
        else:
            print(ds1)                  

In [ ]:
for i in range(10):
    times = np.arange(3000*i, 3000*(i+1))
    fig, ax = plt.subplots(1,1, dpi = 120, figsize = (15,5))
    ax.plot(hd_of1[times], lw = 3, c = [0.2,0.2,0.2])
    ax.plot(coords_mod2[times,0], lw = 3, c = [0.,0.8,0.8])
    ax.spines[['bottom', 'left']].set_linewidth(4)
    ax.tick_params(width=4, length=10)
    ax.set_xticks([0, 1500,3000], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    #ax.set_aspect(1/ax.get_data_ratio())

    ax.spines[['right', 'top']].set_visible(False)
    plt.show()

In [ ]:
inds, inds_label =  get_coord_distribution(coords_mod1, numbins = 50,epsilon = 0.5, metric = 'euclidean', startindex = -1)

meanval_inds = np.zeros((num_neurons, len(inds)))
for neurid in range(num_neurons):
    for ii, i in enumerate(inds):
        meanval_inds[neurid, ii] = np.mean(sspikes_of[inds_label[i],mod_ind1s[neurid]])

In [ ]:
for nn in range(num_neurons):
    print(nn)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 150, alpha = 0.8,
              c = meanval_inds[nn,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
    plt.show()
    #ax.axis('off')

In [ ]:
from matplotlib import gridspec
conj_cells = [0,16,25,29,2,8,19, 49]
numfigs = 3
numw = 1
num_neurons = len(conj_cells)
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 160)
nw = 0
numbins1 = 30
#torsort = np.flip(np.argsort(hd_info))
for nn, n in enumerate(conj_cells):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .4)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], 
                                               width_ratios = [1,1,1],
                                               hspace = 0.2,wspace = .2)
        nw += 1
    xnum = 0

    ax = plt.subplot(gs2[nnn,xnum]) 
    xnum += 1
    mtot, __, __, circ  = binned_statistic_2d(xx1,
                                          yy1,
                                          sspikes_of[:,mod_ind1s[n]], 
                                          statistic = 'mean', 
                                          bins = 30,
                                          expand_binnumbers = True)

    nans = np.isnan(mtot)
    mtot[nans] = np.mean(mtot[~nans])
    mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    vals = np.unique(mtot)
    maxtot = np.sort(mtot.flatten())
    mintot = maxtot[int(0.025*len(maxtot))]
    maxtot = maxtot[int(0.975*len(maxtot))]
    mtot[nans] = np.nan
    
    ax.imshow(mtot, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = mintot, vmax = maxtot)
    ax.set_xticks([])
    ax.set_yticks([])

    ax = plt.subplot(gs2[nnn,xnum], projection = 'polar')
    
    xnum += 1    
    bins = np.linspace(0,2*np.pi,31)

#    rec = binned_statistic(coords_mod1[:,0], sspikes_of[:,mod_ind1s[n]], statistic = 'mean', bins=bins, range=None)[0]
#    rec[np.isnan(rec)] = 0
#    rec /= np.max(rec)
#    rec = gaussian_filter1d(rec, mode = 'wrap', sigma = 1)
    
    dec = binned_statistic(hd_of1[:], sspikes_of[:,mod_ind1s[n]], statistic = 'mean', bins=bins, range=None)[0]
    dec[np.isnan(dec)] = 0
    dec /= np.max(dec)
    dec = gaussian_filter1d(dec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([dec[-1]], dec)), 
            lw = 4, ls = '-', alpha = 0.9, c = [0.2,0.2,0.2], label = 'Dec OF')

    ax.set_rticks([0.33, 0.66,1]) 
    ax.set_yticklabels('')
    ax.set_xticklabels('')
    ax.set_rlim([0,1.1])
    

    ax = plt.subplot(gs2[nnn,xnum], projection = '3d')
    xnum += 1
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 10, alpha = 0.8,
              c = meanval_inds[n,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
#    ax.set_aspect('equal')


In [ ]:

from matplotlib import gridspec
numfigs = 3
numw = 1
num_neurons = len(conj_cells)
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 160)
nw = 0
numbins1 = 30
#torsort = np.flip(np.argsort(hd_info))
for nn, n in enumerate(conj_cells):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .4)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], 
                                               width_ratios = [1,1,1],
                                               hspace = 0.,wspace = 0.)
        nw += 1
    xnum = 0


    ax = plt.subplot(gs2[nnn,xnum], projection = '3d')
    xnum += 1
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 10, alpha = 0.8,
              c = meanval_inds[n,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
#    ax.set_aspect('equal')


## Rat S

In [ ]:
f = np.load('/Users/erihe/OneDrive - NTNU/Prosjekt/Toroidal_topology_grid_cell_data/rat_s_grid_modules_1.npz', 
            allow_pickle = True)
spikes = {}
it = 0
for k in ['spikes_mod1',]:# 'spikes_mod2', 'spikes_mod3']:
    spktemp = f[k][()]
    for kk in spktemp:
        spikes[it] = spktemp[kk]
        it += 1
t = f['t']
x = f['x']
y = f['y']
hd_azimuth = f['azimuth']
f.close()

In [ ]:
min_of_1, max_of_1 = 9939, 12363
min_w_1, max_w_1 = 13670, 14847 
min_w_2, max_w_2 = 14942, 23133 
min_w_3, max_w_3 = 23186, 24936 


In [ ]:
num_neurons = len(spikes)
meanRate_of = np.zeros((num_neurons))
for i in range(num_neurons):
    spk = spikes[i].copy()
    meanRate_of[i] = np.sum((spk>min_of_1) & (spk<max_of_1))
meanRate_of/=(max_of_1-min_of_1)


In [ ]:
hd_azimuth[np.isnan(hd_azimuth)] = 0

In [ ]:
tt, xx, yy, speed, hd_of = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_of_1, max_time = max_of_1, dt_orig = 0.1, res = 10000)


tt1, xx1, yy1, speed1, hd_of1 = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_of_1, max_time = max_of_1, dt_orig = 0.01, res = 100000)

tt_w, xx_w, yy_w, speed_w, hd_w = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_w_1, max_time = max_w_1, dt_orig = 0.1, res = 10000)


tt1_w, xx1_w, yy1_w, speed1_w, hd_w1 = get_pos(x, y, t, hd_azimuth,
                                   min_time = min_w_1, max_time = max_w_1, dt_orig = 0.01, res = 100000)


In [ ]:
#### 'firing_rate' is more temporally precise, but might be slower

t0 = time.time()
sspikes_of, __ = firing_rate(spikes, sigma = 1, min_time = min_of_1, max_time = max_of_1, 
                             dt_orig = 0.1, res = 100000)
print(time.time()- t0)
sspikes_of = np.sqrt(sspikes_of)
t0 = time.time()
Xcorr_of =  cross_corr_dist(sspikes_of, lencorr = 30)
print(time.time()-t0)
#np.savez('roger_of', sspikes = sspikes_of, Xcorr = Xcorr_of)
#del sspikes_of

In [ ]:
#### 'firing_rate' is more temporally precise, but might be slower

t0 = time.time()
sspikes_w1, __ = firing_rate(spikes, sigma = 1, min_time = min_w_1, max_time = max_w_1, 
                             dt_orig = 0.1, res = 100000)
print(time.time()- t0)
sspikes_w1 = np.sqrt(sspikes_w1)
t0 = time.time()
Xcorr_w1 =  cross_corr_dist(sspikes_w1, lencorr = 30)
print(time.time()-t0)
#np.savez('roger_w', sspikes = sspikes_w, Xcorr = Xcorr_w)
#del sspikes_w

In [ ]:
#### 'firing_rate' is more temporally precise, but might be slower

t0 = time.time()
sspikes_w2, __ = firing_rate(spikes, sigma = 1, min_time = min_w_2, max_time = max_w_2, 
                             dt_orig = 0.1, res = 100000)
print(time.time()- t0)
sspikes_w2 = np.sqrt(sspikes_w2)
t0 = time.time()
Xcorr_w2 =  cross_corr_dist(sspikes_w2, lencorr = 30)
print(time.time()-t0)
#np.savez('roger_w', sspikes = sspikes_w, Xcorr = Xcorr_w)
#del sspikes_w

In [ ]:
#### 'firing_rate' is more temporally precise, but might be slower

t0 = time.time()
sspikes_w3, __ = firing_rate(spikes, sigma = 1, min_time = min_w_3, max_time = max_w_3, 
                             dt_orig = 0.1, res = 100000)
print(time.time()- t0)
sspikes_w3 = np.sqrt(sspikes_w3)
t0 = time.time()
Xcorr_w3 =  cross_corr_dist(sspikes_w3, lencorr = 30)
print(time.time()-t0)
#np.savez('roger_w', sspikes = sspikes_w, Xcorr = Xcorr_w)
#del sspikes_w

In [ ]:
del sspikes_w1, sspikes_w2,sspikes_w3

In [ ]:
np.savez('shane_Xcorrs',Xcorr_of = Xcorr_of, Xcorr_w1 = Xcorr_w1 ,Xcorr_w2 = Xcorr_w2, Xcorr_w3 = Xcorr_w3 )

f = np.load('shane_Xcorrs.npz', allow_pickle = True)
Xcorr_of = f['Xcorr_of']
Xcorr_w1 = f['Xcorr_w1']
Xcorr_w2 = f['Xcorr_w2']
Xcorr_w3 = f['Xcorr_w3']
f.close()

In [ ]:
d1 = np.zeros((len(spikes),len(spikes)))
for x in [Xcorr_w1,Xcorr_w2,  Xcorr_of]:    #Xcorr_w3
    dd1 = squareform(pdist(np.square(x), 'correlation'))
    d1 += dd1
infs = np.isinf(d1)
d1[infs] = np.max(d1[~infs])*2
d1 /= 4

In [ ]:
num_neurons = len(spikes)
hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    mtot, __,  circ  = binned_statistic(hd_of, sspikes_of[:,i],
                                              statistic = 'mean', 
                                              bins = 30)
    mu = np.mean(sspikes_of[:,i])
    hd_info[i] = information_score_1d(mtot, circ-1, mu)


In [ ]:
plt.viridis()
dvals = d1[np.triu_indices(len(d1),1)]
thr = np.round(np.percentile(dvals,42),1)
thr = 0.4
print(thr)
ind1 = get_ind(d1,thr, linkage = 'average')

print(np.bincount(isall['is_conj_S1'][ind1 == np.argmax(np.bincount(ind1))]))
print(np.bincount(isall['is_conj_S1'][ind1 == np.argmax(np.bincount(ind1))])/np.max(np.bincount(ind1)))

In [ ]:
fig, ax = plt.subplots(1,1)
ax.scatter(meanRate_of[isall['is_conj_S1'] == 0], hd_info[isall['is_conj_S1'] == 0])
ax.scatter(meanRate_of[isall['is_conj_S1'] != 0], hd_info[isall['is_conj_S1'] != 0])
ax.spines[['bottom', 'left']].set_linewidth(4)
ax.tick_params(width=4, length=10)
ax.set_xticks([0, 3.5,7], [])
ax.set_yticks([0, 0.5,1], [])
ax.set_aspect(1/ax.get_data_ratio())
ax.set_alpha(0)
ax.patch.set_alpha(0)

ax.spines[['right', 'top']].set_visible(False)

In [ ]:

dim = 8         # number of principal components
k = 1000          # number of neighbours for downsampling
nbs = 1000         # number of neighbours for metric
maxdim = 1        # dimension of homology - often just do 1 as it could be expensive (depends on number of points and neighbours)
metric = 'cosine' # what metric to use for persistence
pthr = 0
dt_curr = 0.01
time_resolution = 100000

for sigma in [6,]:
    sspikes_of = firing_rate(spikes, sigma = sigma, min_time = min_of_1, max_time = max_of_1, 
                              dt_orig = dt_curr, res = time_resolution)[0]

    for sp in [0.1]:
        for eps in [0.8,  ]:
            i = np.argmax(np.bincount(ind1))
            mod_ind1s = np.where(ind1 == i)[0]            
            ##########    Initial preprocessing ########## 
            for nn in [5,]:
                n_points = 2500
                sspikes1 = np.sqrt(sspikes_of[:, mod_ind1s])
                num_times_all, num_neurons = np.shape(sspikes1)
                
                movetimes0 = np.arange(0,num_times_all,nn)
                movetimes0 = movetimes0[np.where((speed1[movetimes0]>sp))[0]]
                spksum = np.sum(sspikes1,1)[movetimes0]
                movetimes0 = movetimes0[np.where(spksum> np.percentile(spksum,pthr))[0]]
                sspikes1 = sspikes1[movetimes0, :]

                ##########    Dimension reduce ##########            
                dim_red_spikes_move_scaled = preprocessing.scale(sspikes1,axis = 0)
                dim_red_spikes_move_scaled, e1, e2, var_exp = pca(dim_red_spikes_move_scaled, dim = dim)
                dim_red_spikes_move_scaled /= np.sqrt(e2[:dim])

                fig, ax = plt.subplots(1,1)
                ax.plot(var_exp[:15])
                ax.set_aspect(1/ax.get_data_ratio())
                plt.show()

                ##########    Downsample ##########            
                startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled),1))
                movetimes1 = radial_downsampling(dim_red_spikes_move_scaled, epsilon = eps, 
                    startindex = startindex)
                indstemp  = sample_denoising(dim_red_spikes_move_scaled[movetimes1,:],  k, 
                                                   n_points, 1, 'cosine')[0]
                indstemp = movetimes1[indstemp]
                dim_red_spikes_move_scaled = dim_red_spikes_move_scaled[indstemp,:]
                for n_points in [2100,]:
                        
                    print('sigma', sigma)
                    print('n_points', n_points)
                    print('eps', eps)
                    print('nn', nn)
                    print('sp', sp)
                    indstemp = indstemp[:n_points]
                    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled[:n_points,:]

                    X = squareform(pdist(dim_red_spikes_move_scaled[:,:], metric))

                    ##########    Compute cohomology ##########            
                    persistence, dgms = compute_persistence(X, maxdim)
                    plt.figure()
                    plot_diagrams(dgms, list(np.arange(maxdim+1)), lifetime = True)
                    plt.show()
                    fig = plot_barcode(dgms)

                    ##########    Get coordinates sample ##########            
                    print('movetimes1', len(movetimes1))
                    npoints = len(indstemp)
                    try:
                        coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1,2], bConsistent = True)
                    except:
                        print('except')
                        print('')
                        continue
                    fig, ax = plt.subplots(2,4, figsize = (10,5), dpi = 120)
                    for i in range(3):
                        ax[0, i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
                    ax[0, 3].scatter(*coords_ds[:2,:], s = 100)
                    for i in range(4):
                        ax[0, i].set_aspect(1/ax[0, i].get_data_ratio())

                    for i in range(3):
                        ax[1, i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
                    ax[1, 3].scatter(*coords_ds_consistent[:2,:], s = 100)
                    for i in range(4):
                        ax[1, i].set_aspect(1/ax[1, i].get_data_ratio())


                    ##########    Get coordinates all   ##########            
                    coords_mod0 = get_coords_all(sspikes1, 
                                                 coords_ds_consistent,
                                                 np.arange(len(sspikes1)),                             
                                                 indstemp,
                                                 dim = dim, 
                                                 bPCA = True,
                                                 bPred = False)
                    coords_mod0 = coords_mod0%(2*np.pi)
                    fig, axs = plt.subplots(2,3, figsize = (10,5), dpi = 120)
                    for c in [0,1,2]:
                        coords_mod0tmp = 2*np.pi-coords_mod0[:,c]                        
                        dstmp = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod0tmp)), 
                                        np.mean(np.cos(hd_of1[movetimes0]-coords_mod0tmp)))
                        coords_mod0tmp = (coords_mod0tmp+dstmp)%(2*np.pi)


                        ds = np.arctan2(np.mean(np.sin(hd_of1[movetimes0]-coords_mod0[:,c])), 
                                        np.mean(np.cos(hd_of1[movetimes0]-coords_mod0[:,c])))
                        coords_mod0[:,c] = (coords_mod0[:,c]+ds)%(2*np.pi)

                        ds0 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod0tmp), 
                                              np.cos(hd_of1[movetimes0]-coords_mod0tmp))))
                        ds1 = np.mean(np.abs(np.arctan2(np.sin(hd_of1[movetimes0]-coords_mod0[:,c]), 
                                              np.cos(hd_of1[movetimes0]-coords_mod0[:,c]))))

                        if ds0 < ds1:
                            coords_mod0[:,c] = coords_mod0tmp
                            print(ds0)
                        else:
                            print(ds1)                               

                        print()
                        axs[0,c].plot((coords_mod0[:1000,c]))
                        #plt.plot(hd_of1[movetimes0][:10000])
                        axs[0,c].plot(hd_of1[movetimes0][:1000])
                        nans0 = ~np.isnan(coords_mod0[:,c])
                        mtot, __, __, circ  = binned_statistic_2d(xx1[movetimes0],
                                                                  yy1[movetimes0],
                                                                  coords_mod0[:,c], 
                                                                  statistic = circmean, 
                                                                  bins = 50,
                                                                  expand_binnumbers = True)

                        nans = np.isnan(mtot)
                        sintot = np.sin(mtot)
                        costot = np.cos(mtot)
                        sintot[nans] = np.mean(sintot[~nans])
                        costot[nans] = np.mean(costot[~nans])
                        sintot = gaussian_filter(sintot,1)
                        costot = gaussian_filter(costot,1)
                        mtot = np.cos(np.arctan2(sintot, costot))
                        mtot = gaussian_filter(mtot,1)
                        plt.viridis()
                        mtot[nans] = np.nan
                        axs[1,c].imshow(mtot)
                        axs[1,c].axis('off')
                        axs[1,c].set_aspect(1/axs[1,c].get_data_ratio())

                    plt.show()


In [ ]:

##########    Get coordinates all   ##########            
coords_mod1 = get_coords_all(np.sqrt(sspikes_of[:, ind1==np.argmax(np.bincount(ind1))]),
                             coords_ds_consistent,
                             movetimes0,                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)
coords_mod1 = coords_mod1%(2*np.pi)

In [ ]:
inds, inds_label =  get_coord_distribution(coords_mod1, numbins = 50,epsilon = 0.5, metric = 'euclidean', startindex = -1)
num_neurons = len(mod_ind1s)
meanval_inds = np.zeros((num_neurons, len(inds)))
for neurid in range(num_neurons):
    for ii, i in enumerate(inds):
        meanval_inds[neurid, ii] = np.mean(sspikes_of[inds_label[i],mod_ind1s[neurid]])

In [ ]:
for nn in range(num_neurons):
    print(nn)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 150, alpha = 0.8,
              c = meanval_inds[nn,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
    plt.show()
    #ax.axis('off')

In [ ]:
from matplotlib import gridspec
conj_cells = [17,30,33,34,37,41]
numfigs = 3
numw = 1
num_neurons = len(conj_cells)
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 160)
nw = 0
numbins1 = 30
#torsort = np.flip(np.argsort(hd_info))
for nn, n in enumerate(conj_cells):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .4)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], 
                                               width_ratios = [1,1,1],
                                               hspace = 0.2,wspace = .2)
        nw += 1
    xnum = 0

    ax = plt.subplot(gs2[nnn,xnum]) 
    xnum += 1
    mtot, __, __, circ  = binned_statistic_2d(xx1,
                                          yy1,
                                          sspikes_of[:,mod_ind1s[n]], 
                                          statistic = 'mean', 
                                          bins = 30,
                                          expand_binnumbers = True)

    nans = np.isnan(mtot)
    mtot[nans] = np.mean(mtot[~nans])
    mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    vals = np.unique(mtot)
    maxtot = np.sort(mtot.flatten())
    mintot = maxtot[int(0.025*len(maxtot))]
    maxtot = maxtot[int(0.975*len(maxtot))]
    mtot[nans] = np.nan
    
    ax.imshow(mtot, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = mintot, vmax = maxtot)
    ax.set_xticks([])
    ax.set_yticks([])

    ax = plt.subplot(gs2[nnn,xnum], projection = 'polar')
    
    xnum += 1    
    bins = np.linspace(0,2*np.pi,31)

#    rec = binned_statistic(coords_mod1[:,0], sspikes_of[:,mod_ind1s[n]], statistic = 'mean', bins=bins, range=None)[0]
#    rec[np.isnan(rec)] = 0
#    rec /= np.max(rec)
#    rec = gaussian_filter1d(rec, mode = 'wrap', sigma = 1)
    
    dec = binned_statistic(hd_of1[:], sspikes_of[:,mod_ind1s[n]], statistic = 'mean', bins=bins, range=None)[0]
    dec[np.isnan(dec)] = 0
    dec /= np.max(dec)
    dec = gaussian_filter1d(dec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([dec[-1]], dec)), 
            lw = 4, ls = '-', alpha = 0.9, c = [0.2,0.2,0.2], label = 'Dec OF')

    ax.set_rticks([0.33, 0.66,1]) 
    ax.set_yticklabels('')
    ax.set_xticklabels('')
    ax.set_rlim([0,1.1])
    

    ax = plt.subplot(gs2[nnn,xnum], projection = '3d')
    xnum += 1
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 10, alpha = 0.8,
              c = meanval_inds[n,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
#    ax.set_aspect('equal')


In [ ]:

from matplotlib import gridspec
numfigs = 3
numw = 1
num_neurons = len(conj_cells)
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 160)
nw = 0
numbins1 = 30
#torsort = np.flip(np.argsort(hd_info))
for nn, n in enumerate(conj_cells):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .4)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], 
                                               width_ratios = [1,1,1],
                                               hspace = 0.,wspace = 0.)
        nw += 1
    xnum = 0


    ax = plt.subplot(gs2[nnn,xnum], projection = '3d')
    xnum += 1
    ax.scatter(coords_mod1[inds,0],coords_mod1[inds,1],coords_mod1[inds,2], s = 10, alpha = 0.8,
              c = meanval_inds[n,:])
    ax.set_xticks([0, np.pi,2*np.pi], [])
    ax.set_yticks([0, np.pi,2*np.pi], [])
    ax.set_zticks([0, np.pi,2*np.pi], [])
#    ax.set_aspect('equal')
